<a href="https://colab.research.google.com/github/soumya0422/FitPulse-Health-Anomaly-Detection-from-Fitness-Devices-/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [232]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [314]:
import pandas as pd
import numpy as np

hr_sec = pd.read_csv('/content/drive/MyDrive/Data/heartrate_seconds_merged.csv')
steps_hr = pd.read_csv('/content/drive/MyDrive/Data/hourlySteps_merged.csv')
sleep_d= pd.read_csv('/content/drive/MyDrive/Data/sleepDay_merged.csv')

In [315]:
hr_sec

,Id,Time,Value
0,2022484408,4/12/2016 7:21:00 AM,97
1,2022484408,4/12/2016 7:21:05 AM,102
2,2022484408,4/12/2016 7:21:10 AM,105
3,2022484408,4/12/2016 7:21:20 AM,103
4,2022484408,4/12/2016 7:21:25 AM,101
...,...,...,...
2483653,8877689391,5/12/2016 2:43:53 PM,57
2483654,8877689391,5/12/2016 2:43:58 PM,56
2483655,8877689391,5/12/2016 2:44:03 PM,55
2483656,8877689391,5/12/2016 2:44:18 PM,55


In [316]:
steps_hr

,Id,ActivityHour,StepTotal
0,1503960366,4/12/2016 12:00:00 AM,373
1,1503960366,4/12/2016 1:00:00 AM,160
2,1503960366,4/12/2016 2:00:00 AM,151
3,1503960366,4/12/2016 3:00:00 AM,0
4,1503960366,4/12/2016 4:00:00 AM,0
...,...,...,...
22094,8877689391,5/12/2016 10:00:00 AM,514
22095,8877689391,5/12/2016 11:00:00 AM,1407
22096,8877689391,5/12/2016 12:00:00 PM,3135
22097,8877689391,5/12/2016 1:00:00 PM,307


In [317]:
sleep_d

,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016 12:00:00 AM,1,327,346
1,1503960366,4/13/2016 12:00:00 AM,2,384,407
2,1503960366,4/15/2016 12:00:00 AM,1,412,442
3,1503960366,4/16/2016 12:00:00 AM,2,340,367
4,1503960366,4/17/2016 12:00:00 AM,1,700,712
...,...,...,...,...,...
408,8792009665,4/30/2016 12:00:00 AM,1,343,360
409,8792009665,5/1/2016 12:00:00 AM,1,503,527
410,8792009665,5/2/2016 12:00:00 AM,1,415,423
411,8792009665,5/3/2016 12:00:00 AM,1,516,545


In [318]:
hr_sec['Time'] = pd.to_datetime(hr_sec['Time'])
hr_sec['Time'] = hr_sec['Time'].dt.floor('h')
hr_1H = (
    hr_sec
    .groupby(['Id', 'Time'], as_index=False)
    .agg(heart_rate=('Value', 'mean'))
)
hr_1H['Time'] = pd.to_datetime(
    hr_1H['Time'],
    format='%Y-%m-%d %H:%M:%S', utc=True
    )
hr_1H

,Id,Time,heart_rate
0,2022484408,2016-04-12 07:00:00+00:00,83.200000
1,2022484408,2016-04-12 08:00:00+00:00,68.562005
2,2022484408,2016-04-12 09:00:00+00:00,66.404700
3,2022484408,2016-04-12 10:00:00+00:00,106.716075
4,2022484408,2016-04-12 11:00:00+00:00,67.767157
...,...,...,...
6008,8877689391,2016-05-12 10:00:00+00:00,68.733503
6009,8877689391,2016-05-12 11:00:00+00:00,72.030108
6010,8877689391,2016-05-12 12:00:00+00:00,87.614719
6011,8877689391,2016-05-12 13:00:00+00:00,64.737705


In [319]:
sleep_d['SleepDay'] = pd.to_datetime(
    sleep_d['SleepDay'],
    format='%m/%d/%Y %I:%M:%S %p', utc=True
    )
sleep_d['date'] = sleep_d['SleepDay'].dt.date
sleep_d

,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,date
0,1503960366,2016-04-12 00:00:00+00:00,1,327,346,2016-04-12
1,1503960366,2016-04-13 00:00:00+00:00,2,384,407,2016-04-13
2,1503960366,2016-04-15 00:00:00+00:00,1,412,442,2016-04-15
3,1503960366,2016-04-16 00:00:00+00:00,2,340,367,2016-04-16
4,1503960366,2016-04-17 00:00:00+00:00,1,700,712,2016-04-17
...,...,...,...,...,...,...
408,8792009665,2016-04-30 00:00:00+00:00,1,343,360,2016-04-30
409,8792009665,2016-05-01 00:00:00+00:00,1,503,527,2016-05-01
410,8792009665,2016-05-02 00:00:00+00:00,1,415,423,2016-05-02
411,8792009665,2016-05-03 00:00:00+00:00,1,516,545,2016-05-03


In [320]:
steps_hr['ActivityHour'] = pd.to_datetime(
    steps_hr['ActivityHour'],
    format='%m/%d/%Y %I:%M:%S %p', utc=True
    )
steps_hr['date'] = steps_hr['ActivityHour'].dt.date
steps_hr

,Id,ActivityHour,StepTotal,date
0,1503960366,2016-04-12 00:00:00+00:00,373,2016-04-12
1,1503960366,2016-04-12 01:00:00+00:00,160,2016-04-12
2,1503960366,2016-04-12 02:00:00+00:00,151,2016-04-12
3,1503960366,2016-04-12 03:00:00+00:00,0,2016-04-12
4,1503960366,2016-04-12 04:00:00+00:00,0,2016-04-12
...,...,...,...,...
22094,8877689391,2016-05-12 10:00:00+00:00,514,2016-05-12
22095,8877689391,2016-05-12 11:00:00+00:00,1407,2016-05-12
22096,8877689391,2016-05-12 12:00:00+00:00,3135,2016-05-12
22097,8877689391,2016-05-12 13:00:00+00:00,307,2016-05-12


In [321]:
df_hr_steps = hr_1H.merge(
    steps_hr,
    left_on=['Id', 'Time'],
    right_on=['Id', 'ActivityHour'],
    how='left'
)
df_hr_steps

,Id,Time,heart_rate,ActivityHour,StepTotal,date
0,2022484408,2016-04-12 07:00:00+00:00,83.200000,2016-04-12 07:00:00+00:00,847.0,2016-04-12
1,2022484408,2016-04-12 08:00:00+00:00,68.562005,2016-04-12 08:00:00+00:00,334.0,2016-04-12
2,2022484408,2016-04-12 09:00:00+00:00,66.404700,2016-04-12 09:00:00+00:00,243.0,2016-04-12
3,2022484408,2016-04-12 10:00:00+00:00,106.716075,2016-04-12 10:00:00+00:00,5243.0,2016-04-12
4,2022484408,2016-04-12 11:00:00+00:00,67.767157,2016-04-12 11:00:00+00:00,323.0,2016-04-12
...,...,...,...,...,...,...
6008,8877689391,2016-05-12 10:00:00+00:00,68.733503,2016-05-12 10:00:00+00:00,514.0,2016-05-12
6009,8877689391,2016-05-12 11:00:00+00:00,72.030108,2016-05-12 11:00:00+00:00,1407.0,2016-05-12
6010,8877689391,2016-05-12 12:00:00+00:00,87.614719,2016-05-12 12:00:00+00:00,3135.0,2016-05-12
6011,8877689391,2016-05-12 13:00:00+00:00,64.737705,2016-05-12 13:00:00+00:00,307.0,2016-05-12


In [322]:
df_hr_steps.drop(['ActivityHour'], axis=1, inplace=True)
df_hr_steps

,Id,Time,heart_rate,StepTotal,date
0,2022484408,2016-04-12 07:00:00+00:00,83.200000,847.0,2016-04-12
1,2022484408,2016-04-12 08:00:00+00:00,68.562005,334.0,2016-04-12
2,2022484408,2016-04-12 09:00:00+00:00,66.404700,243.0,2016-04-12
3,2022484408,2016-04-12 10:00:00+00:00,106.716075,5243.0,2016-04-12
4,2022484408,2016-04-12 11:00:00+00:00,67.767157,323.0,2016-04-12
...,...,...,...,...,...
6008,8877689391,2016-05-12 10:00:00+00:00,68.733503,514.0,2016-05-12
6009,8877689391,2016-05-12 11:00:00+00:00,72.030108,1407.0,2016-05-12
6010,8877689391,2016-05-12 12:00:00+00:00,87.614719,3135.0,2016-05-12
6011,8877689391,2016-05-12 13:00:00+00:00,64.737705,307.0,2016-05-12


In [323]:
df_hr_steps_sleep = df_hr_steps.merge(
    sleep_d,
    on=['Id', 'date'],
    how='left'
)
df_hr_steps_sleep

,Id,Time,heart_rate,StepTotal,date,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,2022484408,2016-04-12 07:00:00+00:00,83.200000,847.0,2016-04-12,NaT,NaN,NaN,NaN
1,2022484408,2016-04-12 08:00:00+00:00,68.562005,334.0,2016-04-12,NaT,NaN,NaN,NaN
2,2022484408,2016-04-12 09:00:00+00:00,66.404700,243.0,2016-04-12,NaT,NaN,NaN,NaN
3,2022484408,2016-04-12 10:00:00+00:00,106.716075,5243.0,2016-04-12,NaT,NaN,NaN,NaN
4,2022484408,2016-04-12 11:00:00+00:00,67.767157,323.0,2016-04-12,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6029,8877689391,2016-05-12 10:00:00+00:00,68.733503,514.0,2016-05-12,NaT,NaN,NaN,NaN
6030,8877689391,2016-05-12 11:00:00+00:00,72.030108,1407.0,2016-05-12,NaT,NaN,NaN,NaN
6031,8877689391,2016-05-12 12:00:00+00:00,87.614719,3135.0,2016-05-12,NaT,NaN,NaN,NaN
6032,8877689391,2016-05-12 13:00:00+00:00,64.737705,307.0,2016-05-12,NaT,NaN,NaN,NaN


In [324]:

df_hr_steps_sleep['sleep_efficiency'] = (
    df_hr_steps_sleep['TotalMinutesAsleep'] / df_hr_steps_sleep['TotalTimeInBed']
)
df_hr_steps_sleep['TotalSleepRecords'] = df_hr_steps_sleep['TotalSleepRecords'].fillna(
    df_hr_steps_sleep['TotalSleepRecords']
)

df_hr_steps_sleep['fragmented_sleep'] = df_hr_steps_sleep['TotalSleepRecords'] > 1

df_hr_steps_sleep['date'] = pd.to_datetime(df_hr_steps_sleep['date'])

In [325]:
df_hr_steps_sleep.shape

(6034, 11)

In [326]:
df_hr_steps_sleep.isna().sum()


,0
Id,0
Time,0
heart_rate,0
StepTotal,7
date,7
SleepDay,2121
TotalSleepRecords,2121
TotalMinutesAsleep,2121
TotalTimeInBed,2121
sleep_efficiency,2121


In [327]:
df_hr_steps_sleep.isnull()

,Id,Time,heart_rate,StepTotal,date,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,sleep_efficiency,fragmented_sleep
0,False,False,False,False,False,True,True,True,True,True,False
1,False,False,False,False,False,True,True,True,True,True,False
2,False,False,False,False,False,True,True,True,True,True,False
3,False,False,False,False,False,True,True,True,True,True,False
4,False,False,False,False,False,True,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...
6029,False,False,False,False,False,True,True,True,True,True,False
6030,False,False,False,False,False,True,True,True,True,True,False
6031,False,False,False,False,False,True,True,True,True,True,False
6032,False,False,False,False,False,True,True,True,True,True,False


In [328]:
df_hr_steps_sleep['date'] = df_hr_steps_sleep['Time'].dt.date
df_hr_steps_sleep['sleep_hours'] = df_hr_steps_sleep['TotalMinutesAsleep'] / 60

# per-user median imputation
df_hr_steps_sleep['sleep_hours'] = (
    df_hr_steps_sleep.groupby('Id')['sleep_hours']
      .transform(lambda x: x.fillna(x.median()))
)

# global fallback
df_hr_steps_sleep['sleep_hours'] = df_hr_steps_sleep['sleep_hours'].fillna(
    df_hr_steps_sleep['sleep_hours'].median()
)
df_hr_steps_sleep['StepTotal'] = df_hr_steps_sleep['StepTotal'].fillna(0)
# The 'SleepDay' column was already dropped in a previous execution or does not exist.
# df_hr_steps_sleep.drop(['SleepDay'], axis=1, inplace=True)
df_hr_steps_sleep['TotalSleepRecords'] = df_hr_steps_sleep['TotalSleepRecords'].fillna(0).astype(int)

df_hr_steps_sleep['TotalMinutesAsleep'] = df_hr_steps_sleep['TotalMinutesAsleep'].fillna(0)
df_hr_steps_sleep['TotalTimeInBed'] = df_hr_steps_sleep['TotalTimeInBed'].fillna(0)

df_hr_steps_sleep

,Id,Time,heart_rate,StepTotal,date,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,sleep_efficiency,fragmented_sleep,sleep_hours
0,2022484408,2016-04-12 07:00:00+00:00,83.200000,847.0,2016-04-12,NaT,0,0.0,0.0,NaN,False,7.05
1,2022484408,2016-04-12 08:00:00+00:00,68.562005,334.0,2016-04-12,NaT,0,0.0,0.0,NaN,False,7.05
2,2022484408,2016-04-12 09:00:00+00:00,66.404700,243.0,2016-04-12,NaT,0,0.0,0.0,NaN,False,7.05
3,2022484408,2016-04-12 10:00:00+00:00,106.716075,5243.0,2016-04-12,NaT,0,0.0,0.0,NaN,False,7.05
4,2022484408,2016-04-12 11:00:00+00:00,67.767157,323.0,2016-04-12,NaT,0,0.0,0.0,NaN,False,7.05
...,...,...,...,...,...,...,...,...,...,...,...,...
6029,8877689391,2016-05-12 10:00:00+00:00,68.733503,514.0,2016-05-12,NaT,0,0.0,0.0,NaN,False,7.05
6030,8877689391,2016-05-12 11:00:00+00:00,72.030108,1407.0,2016-05-12,NaT,0,0.0,0.0,NaN,False,7.05
6031,8877689391,2016-05-12 12:00:00+00:00,87.614719,3135.0,2016-05-12,NaT,0,0.0,0.0,NaN,False,7.05
6032,8877689391,2016-05-12 13:00:00+00:00,64.737705,307.0,2016-05-12,NaT,0,0.0,0.0,NaN,False,7.05


In [329]:
df_hr_steps_sleep['fragmented_sleep'] = df_hr_steps_sleep['fragmented_sleep'].astype(int)
df_hr_steps_sleep

,Id,Time,heart_rate,StepTotal,date,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,sleep_efficiency,fragmented_sleep,sleep_hours
0,2022484408,2016-04-12 07:00:00+00:00,83.200000,847.0,2016-04-12,NaT,0,0.0,0.0,NaN,0,7.05
1,2022484408,2016-04-12 08:00:00+00:00,68.562005,334.0,2016-04-12,NaT,0,0.0,0.0,NaN,0,7.05
2,2022484408,2016-04-12 09:00:00+00:00,66.404700,243.0,2016-04-12,NaT,0,0.0,0.0,NaN,0,7.05
3,2022484408,2016-04-12 10:00:00+00:00,106.716075,5243.0,2016-04-12,NaT,0,0.0,0.0,NaN,0,7.05
4,2022484408,2016-04-12 11:00:00+00:00,67.767157,323.0,2016-04-12,NaT,0,0.0,0.0,NaN,0,7.05
...,...,...,...,...,...,...,...,...,...,...,...,...
6029,8877689391,2016-05-12 10:00:00+00:00,68.733503,514.0,2016-05-12,NaT,0,0.0,0.0,NaN,0,7.05
6030,8877689391,2016-05-12 11:00:00+00:00,72.030108,1407.0,2016-05-12,NaT,0,0.0,0.0,NaN,0,7.05
6031,8877689391,2016-05-12 12:00:00+00:00,87.614719,3135.0,2016-05-12,NaT,0,0.0,0.0,NaN,0,7.05
6032,8877689391,2016-05-12 13:00:00+00:00,64.737705,307.0,2016-05-12,NaT,0,0.0,0.0,NaN,0,7.05


In [330]:
df_hr_steps_sleep['sleep_efficiency'] = (
    df_hr_steps_sleep.groupby('Id')['sleep_efficiency']
      .transform(lambda x: x.fillna(x.median()))
)
df_hr_steps_sleep['sleep_efficiency'] = df_hr_steps_sleep['sleep_efficiency'].fillna(
    df_hr_steps_sleep['sleep_efficiency'].median()
)


In [331]:
df_hr_steps_sleep.drop(['TotalMinutesAsleep'], axis=1, inplace=True)
df_hr_steps_sleep.drop(['TotalTimeInBed'], axis=1, inplace=True)

In [332]:
df_hr_steps_sleep.drop(['date'], axis=1, inplace=True)
df_hr_steps_sleep.drop(['TotalSleepRecords'], axis=1, inplace=True)
df_hr_steps_sleep['daily_sleep_date'] = df_hr_steps_sleep['Time'].dt.date
df_hr_steps_sleep['daily_sleep_date'] = pd.to_datetime(df_hr_steps_sleep['daily_sleep_date'])

In [338]:
df_hr_steps_sleep.drop(['SleepDay'], axis=1, inplace=True)

In [339]:
df_hr_steps_sleep.isna().sum()


,0
Id,0
Time,0
heart_rate,0
StepTotal,0
sleep_efficiency,0
fragmented_sleep,0
sleep_hours,0
daily_sleep_date,0


In [340]:
df_hr_steps_sleep.isnull()

,Id,Time,heart_rate,StepTotal,sleep_efficiency,fragmented_sleep,sleep_hours,daily_sleep_date
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
6029,False,False,False,False,False,False,False,False
6030,False,False,False,False,False,False,False,False
6031,False,False,False,False,False,False,False,False
6032,False,False,False,False,False,False,False,False


In [341]:
print(df_hr_steps_sleep.head(10))

           Id                      Time  heart_rate  StepTotal  \
0  2022484408 2016-04-12 07:00:00+00:00   83.200000      847.0   
1  2022484408 2016-04-12 08:00:00+00:00   68.562005      334.0   
2  2022484408 2016-04-12 09:00:00+00:00   66.404700      243.0   
3  2022484408 2016-04-12 10:00:00+00:00  106.716075     5243.0   
4  2022484408 2016-04-12 11:00:00+00:00   67.767157      323.0   
5  2022484408 2016-04-12 12:00:00+00:00   66.230570      184.0   
6  2022484408 2016-04-12 13:00:00+00:00   83.687500      658.0   
7  2022484408 2016-04-12 14:00:00+00:00   80.577093     2168.0   
8  2022484408 2016-04-12 15:00:00+00:00   68.326923      327.0   
9  2022484408 2016-04-12 16:00:00+00:00   71.021635      539.0   

   sleep_efficiency  fragmented_sleep  sleep_hours daily_sleep_date  
0          0.946789                 0         7.05       2016-04-12  
1          0.946789                 0         7.05       2016-04-12  
2          0.946789                 0         7.05       2016-0

In [342]:
df_hr_steps_sleep.columns

Index(['Id', 'Time', 'heart_rate', 'StepTotal', 'sleep_efficiency',
       'fragmented_sleep', 'sleep_hours', 'daily_sleep_date'],
      dtype='object')

In [343]:
df_hr_steps_sleep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6034 entries, 0 to 6033
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Id                6034 non-null   int64              
 1   Time              6034 non-null   datetime64[ns, UTC]
 2   heart_rate        6034 non-null   float64            
 3   StepTotal         6034 non-null   float64            
 4   sleep_efficiency  6034 non-null   float64            
 5   fragmented_sleep  6034 non-null   int64              
 6   sleep_hours       6034 non-null   float64            
 7   daily_sleep_date  6034 non-null   datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(4), int64(2)
memory usage: 377.3 KB


In [344]:
df_hr_steps_sleep.describe()

,Id,heart_rate,StepTotal,sleep_efficiency,fragmented_sleep,sleep_hours,daily_sleep_date
count,6.034000e+03,6034.000000,6034.000000,6034.000000,6034.000000,6034.000000,6034
mean,5.626567e+09,74.800738,507.165065,0.939931,0.071926,6.479187,2016-04-25 21:23:41.146834688
min,2.022484e+09,43.354037,0.000000,0.851240,0.000000,0.966667,2016-04-12 00:00:00
25%,4.388162e+09,64.249289,19.000000,0.919321,0.000000,6.233333,2016-04-19 00:00:00
50%,5.577150e+09,72.473926,243.000000,0.946789,0.000000,7.050000,2016-04-26 00:00:00
75%,6.962181e+09,83.253049,631.750000,0.959184,0.000000,7.650000,2016-05-03 00:00:00
max,8.877689e+09,161.507273,10554.000000,1.000000,1.000000,12.916667,2016-05-12 00:00:00
std,1.946963e+09,14.650901,836.680337,0.027260,0.258386,2.338828,NaN


In [345]:
df_hr_steps_sleep.dtypes

,0
Id,int64
Time,"datetime64[ns, UTC]"
heart_rate,float64
StepTotal,float64
sleep_efficiency,float64
fragmented_sleep,int64
sleep_hours,float64
daily_sleep_date,datetime64[ns]


In [346]:
df_hr_steps_sleep

,Id,Time,heart_rate,StepTotal,sleep_efficiency,fragmented_sleep,sleep_hours,daily_sleep_date
0,2022484408,2016-04-12 07:00:00+00:00,83.200000,847.0,0.946789,0,7.05,2016-04-12
1,2022484408,2016-04-12 08:00:00+00:00,68.562005,334.0,0.946789,0,7.05,2016-04-12
2,2022484408,2016-04-12 09:00:00+00:00,66.404700,243.0,0.946789,0,7.05,2016-04-12
3,2022484408,2016-04-12 10:00:00+00:00,106.716075,5243.0,0.946789,0,7.05,2016-04-12
4,2022484408,2016-04-12 11:00:00+00:00,67.767157,323.0,0.946789,0,7.05,2016-04-12
...,...,...,...,...,...,...,...,...
6029,8877689391,2016-05-12 10:00:00+00:00,68.733503,514.0,0.946789,0,7.05,2016-05-12
6030,8877689391,2016-05-12 11:00:00+00:00,72.030108,1407.0,0.946789,0,7.05,2016-05-12
6031,8877689391,2016-05-12 12:00:00+00:00,87.614719,3135.0,0.946789,0,7.05,2016-05-12
6032,8877689391,2016-05-12 13:00:00+00:00,64.737705,307.0,0.946789,0,7.05,2016-05-12
